In [4]:
!pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable


In [109]:
import os
import numpy as np
import cv2
from skimage.feature import local_binary_pattern, graycoprops, graycomatrix
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cdist
from skimage.filters import gabor
from sklearn.neighbors import KNeighborsClassifier



In [110]:
num_classes = 24
train_samples_per_class = 20

data_dir = "Outex_TC_00012\images"
train_txt_path = "Outex_TC_00012/001/train.txt"
test_txt_path = "Outex_TC_00012/001/test.txt"
train_size = 480

<>:4: SyntaxWarning: invalid escape sequence '\i'
<>:4: SyntaxWarning: invalid escape sequence '\i'
C:\Users\telat\AppData\Local\Temp\ipykernel_1144\167279313.py:4: SyntaxWarning: invalid escape sequence '\i'
  data_dir = "Outex_TC_00012\images"


In [111]:
def load_images(base_dir):
    images = []
    all_files = sorted([f for f in os.listdir(base_dir) if f.endswith('.bmp')])

    for file in all_files:
        file_path = os.path.join(base_dir, file)

        img = cv2.imread(file_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        images.append(gray)
    
    return np.array(images)

def read_labels(txt_path):
        labels = []
        with open(txt_path, 'r') as f:
            next(f)
            for line in f:
                _, label = line.strip().split(' ')
                labels.append(int(label))
        return labels

def load_labels(train_txt_path, test_txt_path):
    train_labels = read_labels(train_txt_path)
    test_labels = read_labels(test_txt_path)

    return train_labels, test_labels


X = load_images(data_dir)
train_labels, test_labels = load_labels(train_txt_path, test_txt_path)

train_size = len(train_labels)

X_train = X[:train_size]
y_train = train_labels
X_test = X[4800:]
y_test = test_labels

print("Train set size:", len(X_train))
print("Test set size:", len(X_test))



Train set size: 480
Test set size: 4320


In [112]:
def nearest_neighbor_classification(X_train, y_train, X_test, metric='euclidean'):
    distances = cdist(X_test, X_train, metric=metric)
    nearest_neighbors = np.argmin(distances, axis=1)
    predictions = [y_train[idx] for idx in nearest_neighbors]
    return predictions

### **Local binary patterns**

In [113]:
def extract_lbp_features(images, radius=1, n_points=8):
    lbp_features = []
    for img in images:
        lbp = local_binary_pattern(img, n_points, radius, method="uniform")
        n_bins = n_points + 2
        hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
        lbp_features.append(hist)
    
    return np.array(lbp_features)

radius = 5
n_points = 5 * radius

X_train_lbp = extract_lbp_features(X_train, radius=radius, n_points=n_points)
X_test_lbp = extract_lbp_features(X_test, radius=radius, n_points=n_points)

y_pred = nearest_neighbor_classification(X_train_lbp, y_train, X_test_lbp)

accuracy = accuracy_score(y_test, y_pred)
print(f"LBP Classification Accuracy: {accuracy * 100:.2f}%")


LBP Classification Accuracy: 74.58%


### **Gray level co-occurrence matrices**

In [114]:
def extract_glcm_features(images, distances=[1], angles=[0], levels=256, props=['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']):
    glcm_features = []
    for img in images:
        glcm = graycomatrix(img, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)
        
        feature_vector = []
        for prop in props:
            feature = graycoprops(glcm, prop).flatten()
            feature_vector.extend(feature)
        
        glcm_features.append(feature_vector)
    
    return np.array(glcm_features)

distances = [2, 3, 4, 5, 6]
angles = [0, np.pi/6, np.pi/4, np.pi/2, 2*np.pi/3, 3*np.pi/4]

X_train_glcm = extract_glcm_features(X_train, distances=distances, angles=angles)
X_test_glcm = extract_glcm_features(X_test, distances=distances, angles=angles)

y_pred_glcm = nearest_neighbor_classification(X_train_glcm, y_train, X_test_glcm)

accuracy_glcm = accuracy_score(y_test, y_pred_glcm)
print(f"GLCM Classification Accuracy: {accuracy_glcm * 100:.2f}%")



GLCM Classification Accuracy: 52.48%


### **Fourier transform based texture descriptors**

In [115]:
def extract_fourier_features(images, low_freq_cutoff=10):
    fourier_features = []
    for img in images:
        f_transform = np.fft.fft2(img)
        f_shift = np.fft.fftshift(f_transform)
        
        magnitude_spectrum = np.abs(f_shift)

        center = (magnitude_spectrum.shape[0] // 2, magnitude_spectrum.shape[1] // 2)
        low_freq_region = magnitude_spectrum[
            center[0] - low_freq_cutoff:center[0] + low_freq_cutoff,
            center[1] - low_freq_cutoff:center[1] + low_freq_cutoff
        ]
        
        feature_vector = low_freq_region.flatten()
        feature_vector = feature_vector / np.sum(feature_vector)
        fourier_features.append(feature_vector)

    return np.array(fourier_features)

low_freq_cutoff = 26

X_train_fourier = extract_fourier_features(X_train, low_freq_cutoff=low_freq_cutoff)
X_test_fourier = extract_fourier_features(X_test, low_freq_cutoff=low_freq_cutoff)

y_pred_fourier = nearest_neighbor_classification(X_train_fourier, y_train, X_test_fourier)

accuracy_fourier = accuracy_score(y_test, y_pred_fourier)
print(f"Fourier Classification Accuracy: {accuracy_fourier * 100:.2f}%")


Fourier Classification Accuracy: 51.88%


### **Gabor filterbanks**

In [116]:
def extract_gabor_features(images, frequencies=[0.1, 0.2, 0.3], thetas=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    gabor_features = []
    for img in images:
        feature_vector = []
        for theta in thetas:
            for freq in frequencies:
                filt_real, filt_imag = gabor(img, frequency=freq, theta=theta)

                feature_vector.append(filt_real.mean())
                feature_vector.append(filt_real.std())
        gabor_features.append(feature_vector)

    return np.array(gabor_features)

frequencies = [0.1, 0.2, 0.3, 0.5, 1, 5, 10]
thetas = [0, np.pi/6, np.pi/4, np.pi/2, 2*np.pi/3, 3*np.pi/4]

X_train_gabor = extract_gabor_features(X_train, frequencies=frequencies, thetas=thetas)
X_test_gabor = extract_gabor_features(X_test, frequencies=frequencies, thetas=thetas)

y_pred_gabor = nearest_neighbor_classification(X_train_gabor, y_train, X_test_gabor)

accuracy_gabor = accuracy_score(y_test, y_pred_gabor)
print(f"Gabor Classification Accuracy: {accuracy_gabor * 100:.2f}%")


Gabor Classification Accuracy: 59.03%


### **Combinations of the above**

In [117]:
def combine_features(*feature_sets):
    return np.concatenate(feature_sets, axis=1)

X_train_combined = combine_features(X_train_lbp, X_train_glcm, X_train_fourier, X_train_gabor)
X_test_combined = combine_features(X_test_lbp, X_test_glcm, X_test_fourier, X_test_gabor)

y_pred_combined = nearest_neighbor_classification(X_train_combined, y_train, X_test_combined)

accuracy_combined = accuracy_score(y_test, y_pred_combined)
print(f"Combined Classification Accuracy: {accuracy_gabor * 100:.2f}%")


Combined Classification Accuracy: 59.03%


In [118]:
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')

knn.fit(X_train_lbp, y_train)
y_pred_lbp = knn.predict(X_test_lbp)
accuracy_lbp = accuracy_score(y_test, y_pred_lbp)
print(f"LBP Classification Accuracy (sklearn): {accuracy_lbp * 100:.2f}%")

knn.fit(X_train_glcm, y_train)
y_pred_glcm = knn.predict(X_test_glcm)
accuracy_glcm = accuracy_score(y_test, y_pred_glcm)
print(f"GLCM Classification Accuracy (sklearn): {accuracy_glcm * 100:.2f}%")

knn.fit(X_train_fourier, y_train)
y_pred_fourier = knn.predict(X_test_fourier)
accuracy_fourier = accuracy_score(y_test, y_pred_fourier)
print(f"Fourier Classification Accuracy (sklearn): {accuracy_fourier * 100:.2f}%")

knn.fit(X_train_gabor, y_train)
y_pred_gabor = knn.predict(X_test_gabor)
accuracy_gabor = accuracy_score(y_test, y_pred_gabor)
print(f"Gabor Classification Accuracy (sklearn): {accuracy_gabor * 100:.2f}%")

knn.fit(X_train_combined, y_train)
y_pred_combined = knn.predict(X_test_combined)
accuracy_combined = accuracy_score(y_test, y_pred_combined)
print(f"Combined Features Classification Accuracy (sklearn): {accuracy_combined * 100:.2f}%")


LBP Classification Accuracy (sklearn): 74.58%
GLCM Classification Accuracy (sklearn): 52.48%
Fourier Classification Accuracy (sklearn): 51.88%
Gabor Classification Accuracy (sklearn): 59.03%
Combined Features Classification Accuracy (sklearn): 54.03%


In [104]:
def knn_classification(X_train, y_train, X_test, y_test, n_neighbors=3, metric='euclidean'):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy, y_pred

distance_metrics = ['euclidean', 'manhattan', 'minkowski', 'chebyshev']

for metric in distance_metrics:
    for feature_type, X_train, X_test in [
        ('LBP', X_train_lbp, X_test_lbp),
        ('GLCM', X_train_glcm, X_test_glcm),
        ('Fourier', X_train_fourier, X_test_fourier),
        ('Gabor', X_train_gabor, X_test_gabor),
        ('Combined', X_train_combined, X_test_combined),
    ]:
        accuracy, y_pred = knn_classification(X_train, y_train, X_test, y_test, metric=metric)
        print(f'Feature Type: {feature_type}, Distance Metric: {metric}, Accuracy: {accuracy* 100:.2f}')
    print()




Feature Type: LBP, Distance Metric: euclidean, Accuracy: 72.11
Feature Type: GLCM, Distance Metric: euclidean, Accuracy: 51.92
Feature Type: Fourier, Distance Metric: euclidean, Accuracy: 52.01
Feature Type: Gabor, Distance Metric: euclidean, Accuracy: 58.98
Feature Type: Combined, Distance Metric: euclidean, Accuracy: 53.24

Feature Type: LBP, Distance Metric: manhattan, Accuracy: 76.09
Feature Type: GLCM, Distance Metric: manhattan, Accuracy: 52.73
Feature Type: Fourier, Distance Metric: manhattan, Accuracy: 50.14
Feature Type: Gabor, Distance Metric: manhattan, Accuracy: 63.15
Feature Type: Combined, Distance Metric: manhattan, Accuracy: 58.15

Feature Type: LBP, Distance Metric: minkowski, Accuracy: 72.11
Feature Type: GLCM, Distance Metric: minkowski, Accuracy: 51.92
Feature Type: Fourier, Distance Metric: minkowski, Accuracy: 52.01
Feature Type: Gabor, Distance Metric: minkowski, Accuracy: 58.98
Feature Type: Combined, Distance Metric: minkowski, Accuracy: 53.24

Feature Type: LB

In [105]:
# Preprocessing Functions
def ensure_grayscale(images):
    """Ensure all images are 2D grayscale arrays."""
    processed_images = []
    for img in images:
        if len(img.shape) == 3:  # If image has 3 channels, convert to grayscale
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        processed_images.append(img)
    return processed_images


def histogram_equalization(images):
    """Apply histogram equalization to a list of images."""
    processed_images = []
    for img in ensure_grayscale(images):
        if img.dtype != np.uint8:  # Ensure the image is 8-bit
            img = (img * 255).astype(np.uint8)
        processed_images.append(cv2.equalizeHist(img))
    return processed_images


def clahe_equalization(images, clip_limit=2.0, tile_grid_size=(8, 8)):
    """Apply CLAHE equalization to a list of images."""
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    processed_images = []
    for img in ensure_grayscale(images):
        if img.dtype != np.uint8:  # Ensure the image is 8-bit
            img = (img * 255).astype(np.uint8)
        processed_images.append(clahe.apply(img))
    return processed_images

"""
def normalize_images(images):
    normalized_images = []
    for img in ensure_grayscale(images):
        normalized_img = (img - np.min(img)) / (np.max(img) - np.min(img))
        normalized_images.append(normalized_img)
    return normalized_images
"""

# Apply Preprocessing
X_train_he = histogram_equalization(X_train)
X_test_he = histogram_equalization(X_test)

X_train_clahe = clahe_equalization(X_train)
X_test_clahe = clahe_equalization(X_test)
"""
X_train_norm = normalize_images(X_train)
X_test_norm = normalize_images(X_test)
"""
# Re-extract Features and Evaluate
for preprocessed_train, preprocessed_test, label in [
    (X_train_he, X_test_he, "Histogram Equalization"),
    (X_train_clahe, X_test_clahe, "CLAHE"),
]:
    # LBP Features
    X_train_lbp = extract_lbp_features(preprocessed_train, radius=radius, n_points=n_points)
    X_test_lbp = extract_lbp_features(preprocessed_test, radius=radius, n_points=n_points)
    accuracy_lbp, _ = knn_classification(X_train_lbp, y_train, X_test_lbp, y_test, metric='euclidean')
    print(f"Preprocessing: {label}, LBP Accuracy: {accuracy_lbp * 100:.2f}%")

    # GLCM Features
    X_train_glcm = extract_glcm_features(preprocessed_train)
    X_test_glcm = extract_glcm_features(preprocessed_test)
    accuracy_glcm, _ = knn_classification(X_train_glcm, y_train, X_test_glcm, y_test, metric='euclidean')
    print(f"Preprocessing: {label}, GLCM Accuracy: {accuracy_glcm * 100:.2f}%")

    # Fourier Features
    X_train_fourier = extract_fourier_features(preprocessed_train)
    X_test_fourier = extract_fourier_features(preprocessed_test)
    accuracy_fourier, _ = knn_classification(X_train_fourier, y_train, X_test_fourier, y_test, metric='euclidean')
    print(f"Preprocessing: {label}, Fourier Accuracy: {accuracy_fourier * 100:.2f}%")

    # Gabor Features
    X_train_gabor = extract_gabor_features(preprocessed_train)
    X_test_gabor = extract_gabor_features(preprocessed_test)
    accuracy_gabor, _ = knn_classification(X_train_gabor, y_train, X_test_gabor, y_test, metric='euclidean')
    print(f"Preprocessing: {label}, Gabor Accuracy: {accuracy_gabor * 100:.2f}%")

    # Combined Features
    X_train_combined = combine_features(X_train_lbp, X_train_glcm, X_train_fourier, X_train_gabor)
    X_test_combined = combine_features(X_test_lbp, X_test_glcm, X_test_fourier, X_test_gabor)
    accuracy_combined, _ = knn_classification(X_train_combined, y_train, X_test_combined, y_test, metric='euclidean')
    print(f"Preprocessing: {label}, Combined Features Accuracy: {accuracy_combined * 100:.2f}%")


    


Preprocessing: Histogram Equalization, LBP Accuracy: 21.69%
Preprocessing: Histogram Equalization, GLCM Accuracy: 4.17%
Preprocessing: Histogram Equalization, Fourier Accuracy: 18.19%
Preprocessing: Histogram Equalization, Gabor Accuracy: 14.19%
Preprocessing: Histogram Equalization, Combined Features Accuracy: 14.19%
Preprocessing: CLAHE, LBP Accuracy: 10.93%
Preprocessing: CLAHE, GLCM Accuracy: 4.17%
Preprocessing: CLAHE, Fourier Accuracy: 21.81%
Preprocessing: CLAHE, Gabor Accuracy: 10.00%
Preprocessing: CLAHE, Combined Features Accuracy: 10.00%


In [106]:
print(X_train[0])
print(X_train_he[0])

[6.39648438e-02 3.72314453e-02 2.77709961e-02 ... 7.38428963e+01
 1.32020874e+02 7.50573848e+01]
[[ 83]
 [ 50]
 [ 48]
 ...
 [195]
 [180]
 [232]]
